## ANALISIS TRACK BASADO EN PWD

In [1]:
#PASO_1
#Importamos un xml procedente de pwd

from xml.dom.minidom import parse
import xml.dom.minidom
import pandas as pd

path='Mallorca70.3_tp'
DOMTree = xml.dom.minidom.parse(path+".xml")


collection = DOMTree.documentElement

trkpts = collection.getElementsByTagName("sample")
list_track = []

count=0
for trkpt in trkpts:
    count +=1
    if len(trkpt.getElementsByTagName("timeoffset"))==0:
        timeoffset = ''
    else:
        timeoffset = float(trkpt.getElementsByTagName("timeoffset")[0].firstChild.nodeValue)
    
    if len(trkpt.getElementsByTagName("hr"))==0:
        hr = ''
    else:
        hr = float(trkpt.getElementsByTagName("hr")[0].firstChild.nodeValue)
    
    if len(trkpt.getElementsByTagName("spd"))==0:
        spd = ''
    else:
        spd = float(trkpt.getElementsByTagName("spd")[0].firstChild.nodeValue)    
    
    if len(trkpt.getElementsByTagName("pwr"))==0:
        pwr = ''
    else:
        pwr = float(trkpt.getElementsByTagName("pwr")[0].firstChild.nodeValue)

    if len(trkpt.getElementsByTagName("cad"))==0:
        cad = ''
    else:
        cad = float(trkpt.getElementsByTagName("cad")[0].firstChild.nodeValue)

    if len(trkpt.getElementsByTagName("dist"))==0:
        dist = ''
    else:
        dist = float(trkpt.getElementsByTagName("dist")[0].firstChild.nodeValue)

    if len(trkpt.getElementsByTagName("lat"))==0:
        lat = ''
    else:
        lat = float(trkpt.getElementsByTagName("lat")[0].firstChild.nodeValue)

    if len(trkpt.getElementsByTagName("lon"))==0:
        lon = ''
    else:
        lon = float(trkpt.getElementsByTagName("lon")[0].firstChild.nodeValue)

    if len(trkpt.getElementsByTagName("alt"))==0:
        alt = ''
    else:
        alt = float(trkpt.getElementsByTagName("alt")[0].firstChild.nodeValue)    

    if len(trkpt.getElementsByTagName("temp"))==0:
        temp = ''
    else:
        temp = float(trkpt.getElementsByTagName("temp")[0].firstChild.nodeValue)
   
    list_track.append([timeoffset, hr, spd, pwr, cad, dist, lat, lon, alt, temp])

print ("Puntos cargados: %d" %count)

    

result = pd.DataFrame(list_track)
result.columns=["timeoffset", "hr", "spd", "pwr", "cad", "dist", "lat", "lon", "alt", "temp"]
result['prueba'] = path
result.to_excel('Intermedio'+path+'.xlsx')
result.to_excel(path+".xlsx")
print('finish')



Puntos cargados: 0


ValueError: Length mismatch: Expected axis has 0 elements, new values have 10 elements

## ANALISIS TRACK BASADO EN GPX

In [2]:
#PASO_1
#Importamos un xml procedente de gpx

from xml.dom.minidom import parse
import xml.dom.minidom
import pandas as pd

#path='Analisis_Aerodinamica'
path='Entreno_preparatorio'
#path='2017_05_13_Triatlón'
DOMTree = xml.dom.minidom.parse(path+".xml")

collection = DOMTree.documentElement

trkpts = collection.getElementsByTagName("trkpt")
list_track = []

def to_dt(dato):
    t=pd.to_datetime(dato)
    return t.hour*3600+t.minute*60+t.second

count=0

for trkpt in trkpts:
    count +=1
    if trkpt.hasAttribute("lat"):
      lat = float(trkpt.getAttribute("lat"))
    if trkpt.hasAttribute("lon"):
      lon = float(trkpt.getAttribute("lon"))
    
    if len(trkpt.getElementsByTagName("ele"))==0:
        alt = ''
    else:
        alt = float(trkpt.getElementsByTagName("ele")[0].firstChild.nodeValue)
    
    if len(trkpt.getElementsByTagName("time"))==0:
        time = ''
    else:
        time = trkpt.getElementsByTagName("time")[0].firstChild.nodeValue
    
    if len(trkpt.getElementsByTagName("gpxdata:hr"))==0:
        hr = ''
    else:
        hr = float(trkpt.getElementsByTagName("gpxdata:hr")[0].firstChild.nodeValue)
    
    if len(trkpt.getElementsByTagName("gpxdata:cadence"))==0:
        cad = ''
    else:
        cad = float(trkpt.getElementsByTagName("gpxdata:cadence")[0].firstChild.nodeValue)
        
    if len(trkpt.getElementsByTagName("gpxdata:temp"))==0:
        temp = ''
    else:
        temp = float(trkpt.getElementsByTagName("gpxdata:temp")[0].firstChild.nodeValue)
    
    if len(trkpt.getElementsByTagName("gpxdata:distance"))==0:
        dist = ''
    else:
        dist = float(trkpt.getElementsByTagName("gpxdata:distance")[0].firstChild.nodeValue)
    
    if len(trkpt.getElementsByTagName("gpxdata:power"))==0:
        pwr = ''
    else:
        pwr = float(trkpt.getElementsByTagName("gpxdata:power")[0].firstChild.nodeValue)

    if len(trkpt.getElementsByTagName("gpxdata:speed"))==0:
        spd = ''
    else:
        spd = float(trkpt.getElementsByTagName("gpxdata:speed")[0].firstChild.nodeValue)
    
    timeoffset = to_dt(time)
    
    list_track.append([timeoffset, hr, spd, pwr, cad, dist, lat, lon, alt, temp])

result = pd.DataFrame(list_track)
result.columns = ["timeoffset", "hr", "spd", "pwr", "cad", "dist", "lat", "lon", "alt", "temp"]
result['prueba'] = path
print ("Puntos cargados: %d" %count)
result.to_excel('Intermedio'+path+'.xlsx')
print('finish')


Puntos cargados: 3819
finish


In [ ]:
#conda install -c conda-forge geopy

## ENRIQUECEMOS EL FICHERO. COMUN PARA TODOS LOS ORIGENES

In [11]:
import pandas as pd
import math as mt

#path='Analisis_Aerodinamica'
path='Entreno_preparatorio'
#path='2017_05_13_Triatlón'

df = pd.read_excel('Intermedio'+path+'.xlsx')

df2 = df[df['lat'] >0]
df2['pwr'].fillna(0, inplace=True)
#df2.drop(['dist','cad','hr','spd'], axis=1, inplace=True)

from geopy.distance import distance
from geopy import Point
i=1
dist_c=[]
dist_c.append(0)
ele_gain = []
ele_gain.append(0)
d=0
e=0
porc=[]
porc.append(0)
spd_c=[]
spd_c.append(0)
spd_prev=[]
spd_prev.append(0)
ele_prev=[]
ele_prev.append(0)
bearing=[]
bearing.append(0)
timeoffset_prev=[]
timeoffset_prev.append(0)


while i < len(df2):
    #print("Punto %d" %i
    punto1 = Point(df2.iloc[i-1]['lon'], df2.iloc[i-1]['lat'])
    punto2 = Point(df2.iloc[i]['lon'], df2.iloc[i]['lat'])
    dist_c.append(distance(punto1, punto2).m)
    d = distance(punto1, punto2).m
    spd_c.append(float(d/(df2.iloc[i]['timeoffset']-df2.iloc[i-1]['timeoffset'])))
    punto1 = df2.iloc[i-1]['alt']
    punto2 = df2.iloc[i]['alt']
    ele_gain.append((punto2)-(punto1))
    e = punto2-punto1
    porc.append(e*100/d)
    lona = df2.iloc[i-1]['lon']
    lata = df2.iloc[i-1]['lat']
    lonb = df2.iloc[i]['lon']
    latb = df2.iloc[i]['lat']
    X = mt.cos(lonb)*mt.sin(latb-lata)
    Y = mt.cos(lona)*mt.sin(lonb)-mt.sin(lona)*mt.cos(lonb)*mt.cos(latb-lata)
    bearing.append(mt.atan2(X,Y)*57.2957795)
    i+=1
df2['dist_c']=dist_c
df2['ele_gain']=ele_gain
df2['porc']=porc
df2['spd_c']=spd_c
df2['bearing']=bearing
i=1
while i < len(df2):
    spd_prev.append(df2.iloc[i-1]['spd_c'])
    ele_prev.append(df2.iloc[i-1]['alt'])
    timeoffset_prev.append(df2.iloc[i-1]['timeoffset'])
    i+=1
df2['spd_prev']=spd_prev
df2['ele_prev']=ele_prev
df2['timeoffset_prev']=timeoffset_prev


df2.to_excel(path+'_ml.xlsx')

print('finish')


C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in double_scalars


finish


In [12]:
df2

,timeoffset,hr,spd,pwr,cad,dist,lat,lon,alt,temp,prueba,dist_c,ele_gain,porc,spd_c,bearing,spd_prev,ele_prev,timeoffset_prev
0,33753,NaN,3.900000,114.000000,61,14.000000,40.516366,-3.925163,660.000000,26.400000,Entreno_preparatorio,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
1,33756,NaN,4.200000,0.000000,61,20.230769,40.516495,-3.925173,660.000000,26.400000,Entreno_preparatorio,14.369365,0.000000,0.000000,4.789788,-96.247618,0.000000,660.000000,33753
2,33757,NaN,4.141667,0.000000,48,24.166667,40.516536,-3.925178,660.000000,26.400000,Entreno_preparatorio,4.586859,0.000000,0.000000,4.586859,-99.768716,4.789788,660.000000,33756
3,33758,NaN,3.936000,80.000000,27,27.960000,40.516574,-3.925187,660.000000,26.400000,Entreno_preparatorio,4.335995,0.000000,0.000000,4.335995,-108.487677,4.586859,660.000000,33757
4,33759,NaN,3.832000,250.000000,51,32.560000,40.516606,-3.925198,660.000000,26.400000,Entreno_preparatorio,3.756282,0.000000,0.000000,3.756282,-115.886278,4.335995,660.000000,33758
5,33760,NaN,3.900000,170.000000,44,38.000000,40.516633,-3.925216,660.000000,26.400000,Entreno_preparatorio,3.599077,0.000000,0.000000,3.599077,-133.263482,3.756282,660.000000,33759
6,33761,NaN,3.932000,0.000000,30,39.600000,40.516661,-3.925245,660.000000,26.400000,Entreno_preparatorio,4.466935,0.000000,0.000000,4.466935,-145.631384,3.599077,660.000000,33760
7,33762,NaN,4.000000,65.600000,30,44.280000,40.516688,-3.925280,660.000000,26.400000,Entreno_preparatorio,4.895980,0.000000,0.000000,4.895980,-151.347280,4.466935,660.000000,33761
8,33763,NaN,4.000000,203.720000,35,48.280000,40.516713,-3.925317,660.000000,26.400000,Entreno_preparatorio,4.944561,0.000000,0.000000,4.944561,-154.425535,4.895980,660.000000,33762
9,33764,NaN,4.032000,201.000000,45,52.280000,40.516734,-3.925357,660.000000,26.400000,Entreno_preparatorio,5.000384,0.000000,0.000000,5.000384,-159.603050,4.944561,660.000000,33763
